In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

%matplotlib inline

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb

from sklearn.cluster import MiniBatchKMeans, KMeans

from sklearn.model_selection import KFold
from tqdm import tqdm
import gc
import datetime
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import matthews_corrcoef, roc_curve, auc, roc_auc_score, classification_report, confusion_matrix
from sklearn.metrics import f1_score,precision_recall_curve,roc_curve, recall_score,precision_score
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold,GroupKFold,StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler,Normalizer,MinMaxScaler,RobustScaler


import warnings
warnings.filterwarnings("ignore")

pd.options.mode.chained_assignment = None
pd.options.display.max_columns = 999
pd.options.display.max_rows = 500

In [ ]:
Path ='../input/genpact-timeseries-baseline/'
train = pd.read_hdf(Path+'train_proc.hdf')
print('train.shape:',train.shape)
test = pd.read_hdf(Path+'test_proc.hdf')
print('test.shape:',test.shape)

In [ ]:
# Path ='../input/'
# train = pd.read_csv(Path+'train.csv')
# print('train.shape:',train.shape)
# test = pd.read_csv(Path+'test.csv')
# print('test.shape:',test.shape)
# meal = pd.read_csv(Path+'meal_info.csv')
# print('meal.shape:',meal.shape)
# center = pd.read_csv(Path+'fulfilment_center_info.csv')
# print('center.shape:',center.shape)

In [ ]:
print(train.shape)
test.shape

In [ ]:
target_col = 'num_orders'
target =  np.log1p(train[target_col])

**Fit LGBM **

In [ ]:
n_splits = 5

In [ ]:
import time

In [ ]:
def rmsle_error(predictions,labels):
    diffs = np.log(predictions + 1) - np.log(labels + 1)
    squared_diffs = np.square(diffs)
    avg = np.mean(squared_diffs)
    return np.sqrt(avg)
#     return ('RMSLE', np.sqrt(avg))

In [ ]:
def runlgb(train,test,target,param,fold_feats,num_round,foldwisefeats=False):
    start = time.time()

    print(param)
    overall_sel_feats =[]
    overall_imp_df = pd.DataFrame()
    
    oof = np.zeros(len(train))
    predictions = np.zeros(len(test))
    start = time.time()
    valid_scores =[]
    fold_importance_df = pd.DataFrame()
    
    overall_sel_feats=[]
    
    folds = KFold(n_splits=n_splits, shuffle=True, random_state=4590)

    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train,target)):
        print()
        print("fold n°{}".format(fold_))

        tr = train.iloc[trn_idx]
        val = train.iloc[val_idx]
        y_val = target.iloc[val_idx]
        y_tr = target.iloc[trn_idx]
        
        cur_features= fold_feats[fold_]
        print('categorical_feats:',categorical_feats)
        
        trn_data = lgb.Dataset(tr[cur_features], label=y_tr,categorical_feature=categorical_feats)
        val_data = lgb.Dataset(val[cur_features], label=y_val, categorical_feature=categorical_feats)
        
        clf = lgb.train(param, trn_data, num_round, valid_sets = [val_data], verbose_eval=100, early_stopping_rounds = 100)

        #Prediction based on current fold selected features
        oof[val_idx] = clf.predict(val[cur_features], num_iteration=clf.best_iteration)

        if not foldwisefeats:
            fold_importance_df["feature"] = cur_features
            if fold_==0:
                fold_importance_df["importance"] =0
            fold_importance_df["importance"] += clf.feature_importance() / n_splits
        valid_scores+=[clf.best_score['valid_0']['rmse']]
        predictions += clf.predict(test[cur_features], num_iteration=clf.best_iteration) / folds.n_splits

    print('valid scores:',valid_scores)
#     print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))
    cvscore = 100 * rmsle_error(np.expm1(oof), np.expm1(target))
    print("CV score: {:<8.5f}".format(cvscore))

    end = time.time()
    print("LGB Exec Time:",end - start)
        
    return fold_importance_df,predictions,oof

In [ ]:
def ScaleData(train,test):
    scaler = MinMaxScaler()
    scaler.fit(train)
    
    traincols = train.columns
    scaled_train=scaler.transform(train)
    scaled_train = pd.DataFrame(scaled_train)
    scaled_train.columns = traincols

    scaled_test=scaler.transform(test)
    scaled_test = pd.DataFrame(scaled_test)
    scaled_test.columns = traincols
    print(scaled_test.shape)
    print(scaled_train.shape)
    return scaled_train, scaled_test

In [ ]:
# train_scaled,test_scaled=ScaleData(train[sel_feats_kmeans],test[sel_feats_kmeans])

In [ ]:
excluded_cols = ['id',target_col,'week']
features = [col for col in train.columns if col not in excluded_cols]
# categorical_feats =['meal_id','center_id']
categorical_feats =['meal_id','center_id','meal_id_center_id','city_code',
                   'center_type','cuisine','region_code']
print(features)

In [ ]:
# target = np.log(train[target_col]+1)

In [ ]:
# del train[target_col]

In [ ]:
param = {'num_leaves': 110,#31
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': 8,#8,
         'learning_rate': 0.05, #0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

In [ ]:
#Run Permute Importance
fold_feats=[]
# enc_cols = [col for col in val_encs[0].columns if 'targetenc' in col]
for i in range(0,5):
    fold_feats += [features]
# num_round =30000
num_round = 10000
fold_importance_df,predictions,oof = runlgb(train,test,target,param,fold_feats,num_round)

In [ ]:
def submit_file(test,predictions,filename):
    sub_df = pd.DataFrame({"id":test["id"].values})
    sub_df[target_col] = np.expm1(predictions)
    sub_df.to_csv(filename, index=False)
    print(sub_df.head(10))

In [ ]:
def print_bestfeatures(fold_importance_df):
    cols = (fold_importance_df[["feature", "importance"]]
            .groupby("feature")
            .mean()
            .sort_values(by="importance", ascending=False)[:1000].index)

    best_features = fold_importance_df.loc[fold_importance_df.feature.isin(cols)]
    best_features=best_features.sort_values(by="importance", ascending=False)
    print(best_features[:100])

In [ ]:
np.savetxt('LGBM_Meta_simple_all_lagforkeyfeat.npy',oof)

In [ ]:
filename="submission_simple_all_lagforkeyfeat.csv"
submit_file(test,predictions,filename)

In [ ]:
print_bestfeatures(fold_importance_df)